In [1]:
''' Modified from lukefeilberg's code here: https://github.com/lukefeilberg/onion'''
import pandas as pd
import numpy as np
import warnings
import re
import time
import requests
import json
import csv
import datetime
from tqdm import tqdm

In [2]:
dates_list = []

# Making list of dates; Each January 1st and June 1st from 2015 until January 1st 2020
for i in range(10,21):
    dates_list.append('01/01/20'+str(i))
    # dates_list.append('01/02/20'+str(i))
    dates_list.append('01/03/20'+str(i))
    # dates_list.append('01/04/20'+str(i))
    dates_list.append('01/05/20'+str(i))
    # dates_list.append('01/06/20'+str(i))
    dates_list.append('01/07/20'+str(i))
    # dates_list.append('01/08/20'+str(i))
    dates_list.append('01/09/20'+str(i))
    # dates_list.append('01/10/20'+str(i))
    # dates_list.append('01/11/20'+str(i))
    dates_list.append('01/12/20'+str(i))

dates_list = dates_list[:-11]    
dates_list

['01/01/2010',
 '01/03/2010',
 '01/05/2010',
 '01/07/2010',
 '01/09/2010',
 '01/12/2010',
 '01/01/2011',
 '01/03/2011',
 '01/05/2011',
 '01/07/2011',
 '01/09/2011',
 '01/12/2011',
 '01/01/2012',
 '01/03/2012',
 '01/05/2012',
 '01/07/2012',
 '01/09/2012',
 '01/12/2012',
 '01/01/2013',
 '01/03/2013',
 '01/05/2013',
 '01/07/2013',
 '01/09/2013',
 '01/12/2013',
 '01/01/2014',
 '01/03/2014',
 '01/05/2014',
 '01/07/2014',
 '01/09/2014',
 '01/12/2014',
 '01/01/2015',
 '01/03/2015',
 '01/05/2015',
 '01/07/2015',
 '01/09/2015',
 '01/12/2015',
 '01/01/2016',
 '01/03/2016',
 '01/05/2016',
 '01/07/2016',
 '01/09/2016',
 '01/12/2016',
 '01/01/2017',
 '01/03/2017',
 '01/05/2017',
 '01/07/2017',
 '01/09/2017',
 '01/12/2017',
 '01/01/2018',
 '01/03/2018',
 '01/05/2018',
 '01/07/2018',
 '01/09/2018',
 '01/12/2018',
 '01/01/2019']

In [3]:
def getPushshiftData(after, before, sub):
    url = ('https://api.pushshift.io/reddit/search/submission/?size=1000&after='+
           str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort_type=score'+'&sort=desc')
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [4]:
def getTimeStamp(date_input):
    return time.mktime(datetime.datetime.strptime(date_input, "%d/%m/%Y").timetuple())

dates = [int(getTimeStamp(date)) for date in dates_list]
dates

[1262325600,
 1267423200,
 1272690000,
 1277960400,
 1283317200,
 1291183200,
 1293861600,
 1298959200,
 1304226000,
 1309496400,
 1314853200,
 1322719200,
 1325397600,
 1330581600,
 1335848400,
 1341118800,
 1346475600,
 1354341600,
 1357020000,
 1362117600,
 1367384400,
 1372654800,
 1378011600,
 1385877600,
 1388556000,
 1393653600,
 1398920400,
 1404190800,
 1409547600,
 1417413600,
 1420092000,
 1425189600,
 1430456400,
 1435726800,
 1441083600,
 1448949600,
 1451628000,
 1456812000,
 1462078800,
 1467349200,
 1472706000,
 1480572000,
 1483250400,
 1488348000,
 1493614800,
 1498885200,
 1504242000,
 1512108000,
 1514786400,
 1519884000,
 1525150800,
 1530421200,
 1535778000,
 1543644000,
 1546322400]

In [6]:
def getTitles(subreddit):
    titles = []

    for i in tqdm(range(len(dates)-1)):
        # Setting up dates
        after  = dates[i]
        before = dates[i+1]

        # Getting subreddit data between the dates after and before from r/NotTheOnion
        raw_json = getPushshiftData(after,before,subreddit)

        # Extracting just the title
        
        titles_new = [post['title'] for post in raw_json]

        # Appending new data on
        titles = titles + titles_new
        
        time.sleep(1)

    # A few posts were extracted twice, set gets rid of duplicates
    titles = list(set(titles))
    return titles

In [7]:
not_onion = getTitles('nottheonion')
print(len(not_onion))

100%|██████████| 54/54 [02:02<00:00,  2.27s/it]


38841


In [47]:
onion = getTitles('theonion')
print(len(onion))

100%|██████████| 120/120 [03:53<00:00,  1.94s/it]

15293


In [51]:
df1= pd.DataFrame({'text':onion})
df1['label'] = 1
df1 = df1.sample(frac=1).reset_index(drop=True)

df2 = pd.DataFrame({'text':not_onion})
df2['label'] = 0
df2 = df2.sample(frac=1).reset_index(drop=True)

df1_clipped = df1[:15000]
df2_clipped = df2[:15000]

,text,label
0,Comcast Whines About How Net Neutrality Is Lik...,0
1,Chattanooga Wants You to Know That Iron Man 3 ...,0
2,Men who buried dog alive with nail in its head...,0
3,Massive Swirl Of Ocean Debris To Be Designated...,0
4,Half-naked man makes Subway sandwiches in ‘con...,0
...,...,...
14995,Confused Japanese tourists trigger highway pur...,0
14996,"‘Dead’ Teen ‘Wakes Up’ Screaming, Banging Insi...",0
14997,Congresswoman says pornography is a root cause...,0
14998,Russian Woman Eats a Pig to Avoid Paying Child...,0


In [52]:
# Combining both datasets
df = pd.concat([df1_clipped, df2_clipped])

# Shuffling the dataset
df = df.sample(frac=1).reset_index(drop=True)

# Saving the 'uncleaned' dataframe to a csv file
df.to_csv('OnionOrNot_large.csv', index = False)